In [1]:
%run "SparkApp.ipynb"

24/12/21 14:27:54 WARN Utils: Your hostname, Ala resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/21 14:27:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ala/.ivy2/cache
The jars for the packages stored in: /home/ala/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-745fc576-e06a-44e3-ac61-723b50081b54;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution

In [2]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, ArrayType

In [3]:
json_schema = StructType([
    StructField("id", StringType(), True),  # Le champ 'id' est une chaîne
    StructField("tripUpdate", StructType([  # Le champ 'tripUpdate' est un objet imbriqué
        StructField("trip", StructType([  # Le champ 'trip' est aussi un objet imbriqué
            StructField("tripId", StringType(), True),
            StructField("startTime", StringType(), True),
            StructField("startDate", StringType(), True),
            StructField("scheduleRelationship", StringType(), True),
            StructField("routeId", StringType(), True),
            StructField("directionId", LongType(), True)  # Utilisation de LongType pour les entiers longs
        ]), True),
        StructField("stopTimeUpdate", ArrayType(  # Le champ 'stopTimeUpdate' est un tableau
            StructType([  # Chaque élément du tableau est un objet
                StructField("stopSequence", LongType(), True),
                StructField("departure", StructType([  # Le champ 'departure' est un objet imbriqué
                    StructField("time", StringType(), True)
                ]), True),
                StructField("stopId", StringType(), True),
                StructField("scheduleRelationship", StringType(), True)
            ])
        ), True)
    ]), True),
    StructField("timestamp", StringType(), True)  # Le champ 'timestamp' est une chaîne
])


In [17]:
kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "tripupdate") \
    .option("startingOffsets","earliest") \
    .load()

In [18]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [25]:
from pyspark.sql.functions import expr

kafka_json_df = kafka_df.withColumn("value", expr("cast(value as string)"))

In [27]:
from pyspark.sql.functions import from_json,col

streaming_df = kafka_json_df.withColumn("values_json", from_json(col("value"), json_schema)).selectExpr("values_json.*")

In [28]:
streaming_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- tripUpdate: struct (nullable = true)
 |    |-- trip: struct (nullable = true)
 |    |    |-- tripId: string (nullable = true)
 |    |    |-- startTime: string (nullable = true)
 |    |    |-- startDate: string (nullable = true)
 |    |    |-- scheduleRelationship: string (nullable = true)
 |    |    |-- routeId: string (nullable = true)
 |    |    |-- directionId: long (nullable = true)
 |    |-- stopTimeUpdate: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- stopSequence: long (nullable = true)
 |    |    |    |-- departure: struct (nullable = true)
 |    |    |    |    |-- time: string (nullable = true)
 |    |    |    |-- stopId: string (nullable = true)
 |    |    |    |-- scheduleRelationship: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [30]:
from pyspark.sql.functions import explode

exploded_df = streaming_df.withColumn("tripUpdate_stopTimeUpdate", explode("tripUpdate.stopTimeUpdate"))

In [31]:
exploded_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- tripUpdate: struct (nullable = true)
 |    |-- trip: struct (nullable = true)
 |    |    |-- tripId: string (nullable = true)
 |    |    |-- startTime: string (nullable = true)
 |    |    |-- startDate: string (nullable = true)
 |    |    |-- scheduleRelationship: string (nullable = true)
 |    |    |-- routeId: string (nullable = true)
 |    |    |-- directionId: long (nullable = true)
 |    |-- stopTimeUpdate: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- stopSequence: long (nullable = true)
 |    |    |    |-- departure: struct (nullable = true)
 |    |    |    |    |-- time: string (nullable = true)
 |    |    |    |-- stopId: string (nullable = true)
 |    |    |    |-- scheduleRelationship: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- tripUpdate_stopTimeUpdate: struct (nullable = true)
 |    |-- stopSequence: long (nullable = true)
 |    |-- departure: struct (

In [37]:
selected_df = exploded_df.select(
    "id",
    "timestamp",
    "tripUpdate.trip.tripId",
    "tripUpdate.trip.startTime",
    "tripUpdate.trip.startDate",
    "tripUpdate.trip.scheduleRelationship",
    "tripUpdate.trip.routeId",
    "tripUpdate.trip.directionId",
    "exploded_stopTimeUpdate.stopSequence",
    "exploded_stopTimeUpdate.departure.time",
    "exploded_stopTimeUpdate.stopId",
    "exploded_stopTimeUpdate.scheduleRelationship"
)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `exploded_stopTimeUpdate`.`stopSequence` cannot be resolved. Did you mean one of the following? [`tripUpdate_stopTimeUpdate`, `tripUpdate`, `timestamp`, `id`].;
'Project [id#203, timestamp#205, tripUpdate#204.trip.tripId AS tripId#233, tripUpdate#204.trip.startTime AS startTime#234, tripUpdate#204.trip.startDate AS startDate#235, tripUpdate#204.trip.scheduleRelationship AS scheduleRelationship#236, tripUpdate#204.trip.routeId AS routeId#237, tripUpdate#204.trip.directionId AS directionId#238L, 'exploded_stopTimeUpdate.stopSequence, 'exploded_stopTimeUpdate.departure.time, 'exploded_stopTimeUpdate.stopId, 'exploded_stopTimeUpdate.scheduleRelationship]
+- Project [id#203, tripUpdate#204, timestamp#205, tripUpdate_stopTimeUpdate#213]
   +- Generate explode(tripUpdate#204.stopTimeUpdate), false, [tripUpdate_stopTimeUpdate#213]
      +- Project [values_json#194.id AS id#203, values_json#194.tripU

In [34]:
from pyspark.sql.functions import explode

explodedj= streaming_df.withColumn("tripUpdate_trip", explode("tripUpdate.trip"))

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "explode(tripUpdate.trip)" due to data type mismatch: Parameter 1 requires the ("ARRAY" or "MAP") type, however "tripUpdate.trip" has the type "STRUCT<tripId: STRING, startTime: STRING, startDate: STRING, scheduleRelationship: STRING, routeId: STRING, directionId: BIGINT>".;
'Project [id#203, tripUpdate#204, timestamp#205, explode(tripUpdate#204.trip) AS tripUpdate_trip#221]
+- Project [values_json#194.id AS id#203, values_json#194.tripUpdate AS tripUpdate#204, values_json#194.timestamp AS timestamp#205]
   +- Project [key#144, value#185, topic#146, partition#147, offset#148L, timestamp#149, timestampType#150, from_json(StructField(id,StringType,true), StructField(tripUpdate,StructType(StructField(trip,StructType(StructField(tripId,StringType,true),StructField(startTime,StringType,true),StructField(startDate,StringType,true),StructField(scheduleRelationship,StringType,true),StructField(routeId,StringType,true),

In [6]:
from pyspark.sql.functions import expr

kafka_json_df = kafka_df.withColumn("value", expr("cast(value as string)"))

In [20]:
from pyspark.sql.functions import from_json,col

streaming_df = kafka_json_df.withColumn("values_json", from_json(col("value"), json_schema)).selectExpr("values_json.*")

In [8]:
streaming_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- tripUpdate: struct (nullable = true)
 |    |-- trip: struct (nullable = true)
 |    |    |-- tripId: string (nullable = true)
 |    |    |-- startTime: string (nullable = true)
 |    |    |-- startDate: string (nullable = true)
 |    |    |-- scheduleRelationship: string (nullable = true)
 |    |    |-- routeId: string (nullable = true)
 |    |    |-- directionId: long (nullable = true)
 |    |-- stopTimeUpdate: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- stopSequence: long (nullable = true)
 |    |    |    |-- departure: struct (nullable = true)
 |    |    |    |    |-- time: string (nullable = true)
 |    |    |    |-- stopId: string (nullable = true)
 |    |    |    |-- scheduleRelationship: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [12]:
from pyspark.sql.functions import unix_timestamp, explode, col


In [13]:
exploded_df = streaming_df.withColumn("stopTimeUpdateElement", explode(col("tripUpdate.stopTimeUpdate")))

# Sélectionner les champs nécessaires dans 'stopTimeUpdateElement'
flattened_df = exploded_df.select(
    "id",
    "tripUpdate.trip.tripId",
    "tripUpdate.trip.startTime",
    "stopTimeUpdateElement.stopSequence",
    "stopTimeUpdateElement.departure.time",
    "stopTimeUpdateElement.departure.stopId",
    "stopTimeUpdateElement.departure.scheduleRelationship"
)

# Afficher le résultat
flattened_df.show(truncate=False)

AnalysisException: [FIELD_NOT_FOUND] No such struct field `stopId` in `time`.

In [ ]:
exploded_df = streaming_df.withColumn("data_devices", explode("data.devices"))

In [ ]:
from pyspark.sql.functions import col

# Example: Selecting nested fields within tripUpdate
selected_df = streaming_df.select(
    "id",
    "tripUpdate.trip.tripId",
    "tripUpdate.trip.startTime",
    "tripUpdate.trip.startDate",
    "tripUpdate.trip.scheduleRelationship",
    "tripUpdate.trip.routeId",
    "tripUpdate.trip.directionId",
    "tripUpdate.stopTimeUpdate"
)

selected_df.show(truncate=False)

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
kafka

In [74]:
from pyspark.sql import functions as F

In [77]:
df_exploded = streaming_df.withColumn("tripUpdate_exploded", F.explode("tripUpdate"))

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "explode(tripUpdate)" due to data type mismatch: Parameter 1 requires the ("ARRAY" or "MAP") type, however "tripUpdate" has the type "STRUCT<trip: STRUCT<tripId: STRING, startTime: STRING, startDate: STRING, scheduleRelationship: STRING, routeId: STRING, directionId: BIGINT>, stopTimeUpdate: ARRAY<STRUCT<stopSequence: BIGINT, departure: STRUCT<time: STRING>, stopId: STRING, scheduleRelationship: STRING>>>".;
'Project [id#317, tripUpdate#318, timestamp#319, explode(tripUpdate#318) AS tripUpdate_exploded#514]
+- Project [values_json#308.id AS id#317, values_json#308.tripUpdate AS tripUpdate#318, values_json#308.timestamp AS timestamp#319]
   +- Project [key#286, value#300, topic#288, partition#289, offset#290L, timestamp#291, timestampType#292, from_json(StructField(id,StringType,true), StructField(tripUpdate,StructType(StructField(trip,StructType(StructField(tripId,StringType,true),StructField(startTime,StringTy

In [60]:
df_with_delay = df_exploded.withColumn(
    "departure_delay",
    (unix_timestamp("stopTime.departure.time") - unix_timestamp("tripUpdate.trip.startTime")).cast("long")
)

In [63]:
df_with_delay.printSchema()

root
 |-- id: string (nullable = true)
 |-- tripUpdate: struct (nullable = true)
 |    |-- trip: struct (nullable = true)
 |    |    |-- tripId: string (nullable = true)
 |    |    |-- startTime: string (nullable = true)
 |    |    |-- startDate: string (nullable = true)
 |    |    |-- scheduleRelationship: string (nullable = true)
 |    |    |-- routeId: string (nullable = true)
 |    |    |-- directionId: long (nullable = true)
 |    |-- stopTimeUpdate: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- stopSequence: long (nullable = true)
 |    |    |    |-- departure: struct (nullable = true)
 |    |    |    |    |-- time: string (nullable = true)
 |    |    |    |-- stopId: string (nullable = true)
 |    |    |    |-- scheduleRelationship: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- stopTime: struct (nullable = true)
 |    |-- stopSequence: long (nullable = true)
 |    |-- departure: struct (nullable = true)


In [65]:
df_selected_rev = streaming_df.select(
    "id",
    "tripUpdate.trip.tripId",
    "tripUpdate.trip.startTime",
    "tripUpdate.stopTimeUpdate",
    "timestamp"
)

In [70]:
df_exploded = streaming_df.withColumn("stopTimeUpdate_exploded", F.explode("tripUpdate.stopTimeUpdate"))

In [71]:
df_exploded.printSchema()

root
 |-- id: string (nullable = true)
 |-- tripUpdate: struct (nullable = true)
 |    |-- trip: struct (nullable = true)
 |    |    |-- tripId: string (nullable = true)
 |    |    |-- startTime: string (nullable = true)
 |    |    |-- startDate: string (nullable = true)
 |    |    |-- scheduleRelationship: string (nullable = true)
 |    |    |-- routeId: string (nullable = true)
 |    |    |-- directionId: long (nullable = true)
 |    |-- stopTimeUpdate: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- stopSequence: long (nullable = true)
 |    |    |    |-- departure: struct (nullable = true)
 |    |    |    |    |-- time: string (nullable = true)
 |    |    |    |-- stopId: string (nullable = true)
 |    |    |    |-- scheduleRelationship: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- stopTimeUpdate_exploded: struct (nullable = true)
 |    |-- stopSequence: long (nullable = true)
 |    |-- departure: struct (nu

In [72]:
df_flattened = df_exploded.select(
    "id",
    "tripId",
    "startTime",
    "timestamp",
    F.col("stopTimeUpdate_exploded.stopSequence").alias("stopSequence"),
    F.col("stopTimeUpdate_exploded.departure.time").alias("departure_time"),
    F.col("stopTimeUpdate_exploded.stopId").alias("stopId"),
    F.col("stopTimeUpdate_exploded.scheduleRelationship").alias("scheduleRelationship")
)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `tripId` cannot be resolved. Did you mean one of the following? [`id`, `tripUpdate`, `timestamp`, `stopTimeUpdate_exploded`].;
'Project [id#317, 'tripId, 'startTime, timestamp#319, stopTimeUpdate_exploded#498.stopSequence AS stopSequence#503L, stopTimeUpdate_exploded#498.departure.time AS departure_time#504, stopTimeUpdate_exploded#498.stopId AS stopId#505, stopTimeUpdate_exploded#498.scheduleRelationship AS scheduleRelationship#506]
+- Project [id#317, tripUpdate#318, timestamp#319, stopTimeUpdate_exploded#498]
   +- Generate explode(tripUpdate#318.stopTimeUpdate), false, [stopTimeUpdate_exploded#498]
      +- Project [values_json#308.id AS id#317, values_json#308.tripUpdate AS tripUpdate#318, values_json#308.timestamp AS timestamp#319]
         +- Project [key#286, value#300, topic#288, partition#289, offset#290L, timestamp#291, timestampType#292, from_json(StructField(id,StringType,true),

In [64]:
df_with_delay.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start() \
    .awaitTermination()

24/12/21 00:13:23 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c684c155-44f7-410c-8563-9015df3c8413. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/21 00:13:23 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/12/21 00:13:23 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/12/21 00:13:23 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/12/21 00:13:23 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/12/21 00:13:23 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known con

-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/ala/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ala/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 